In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Cognitive_Processing/GSE249399'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Individual Variability in Human Cell Type Transcriptomes and Epigenomes [YM2A]"
!Series_summary	"Diversity and individual variability are essential to human cognitive function. Identifying the conserved and variable (epi)genomic signatures of the brain’s cellular components is critical for understanding the neurobiological basis of individual variation in brain function. We applied single nucleus methylome and transcriptome sequence (snmCT-seq) to neurons from the frontal cortex of 11 adult human donors spanning a range of ages from 23 to 74, including males and females (Broadmann Area BA46). We clustered cells into brain cell types based on methylation features. We then examined the transcriptome and epigenome features in each cell type between and within individual donors. Taking advantage of the multimodal measurements in single cells, we also identified the relation between RNA expression and methylation level.These data with multiomics measur

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # Based on the description, transcriptome sequence data is available

# Check data availability for Cognitive_Processing (trait), age, and gender
# In this particular dataset output, it seems Cognitive_Processing is not explicitly given or inferable, so set it to None
# Age and gender are available directly under their respective keys
age_row = 1
gender_row = 2

# Define data type conversion functions
def convert_trait(value_string):
    try:
        value = value_string.split(':')[1].strip()
        # Placeholder since the trait is not available
        return None
    except:
        return None

def convert_age(value_string):
    try:
        value = value_string.split(':')[1].strip()
        return float(value)  # Age is continuous
    except:
        return None

def convert_gender(value_string):
    try:
        value = value_string.split(':')[1].strip().lower()
        return 1 if value == 'male' else 0  # Male to 1, Female to 0
    except:
        return None

# Save cohort information
save_cohort_info('GSE249399', './preprocessed/Cognitive_Processing/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and save clinical data if trait_row is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Cognitive_Processing', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Cognitive_Processing/trait_data/GSE249399.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
